## BYOL for image segmentation

Example with UNet model from [segmentation-models-pytorch](https://github.com/qubvel/segmentation_models.pytorch/tree/master).

In [1]:
import sys

sys.path.append("./../")

from byol.models import BYOL, UNETModule
from byol.utils import BYOLTransforms1, BYOLTransforms
import torch
import pytorch_lightning as pl
import torchvision
import numpy as np

accelerator = "gpu" if torch.cuda.is_available() else "cpu"

/home/rafael/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from utils.oxford_pet import SimpleOxfordPetDataset, BYOLOxfordPetDataset
import os
from torch.utils.data import DataLoader

root = "."
SimpleOxfordPetDataset.download(root)
# init train, val, test sets
train_dataset = SimpleOxfordPetDataset(root, "train")
valid_dataset = SimpleOxfordPetDataset(root, "valid")
test_dataset = SimpleOxfordPetDataset(root, "test")

print(f"Train size: {len(train_dataset)}")
print(f"Valid size: {len(valid_dataset)}")
print(f"Test size: {len(test_dataset)}")

n_cpu = os.cpu_count()
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=n_cpu)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=n_cpu)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=n_cpu)

Train size: 3312
Valid size: 368
Test size: 3669


### Training an UNET model on the SimpleOxfordPet Dataset

In [3]:
unet_module = UNETModule()

unet_trainer = pl.Trainer(
    max_epochs=1,
    accelerator=accelerator,
    devices=1,
)

unet_trainer.fit(unet_module, train_dataloader, valid_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
unet_trainer.test(unet_module, test_dataloader)

In [5]:
import matplotlib.pyplot as plt

batch = next(iter(test_dataloader))
with torch.no_grad():
    unet_module.eval()
    logits = unet_module(batch["image"].float())
pr_masks = logits.sigmoid()

for image, gt_mask, pr_mask in zip(batch["image"], batch["mask"], pr_masks):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(image.numpy().transpose(1, 2, 0))  # convert CHW -> HWC
    plt.title("Image")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(gt_mask.numpy().squeeze()) # just squeeze classes dim, because we have only one class
    plt.title("Ground truth")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(pr_mask.numpy().squeeze()) # just squeeze classes dim, because we have only one class
    plt.title("Prediction")
    plt.axis("off")

    plt.show()
     

### Training BYOL on SimpleOxfordPet Dataset

In [6]:
transform = BYOLTransforms(
    view1=BYOLTransforms1(input_size=256),
    view2=BYOLTransforms1(input_size=256)
)
byol_dataset = BYOLOxfordPetDataset(root, "test", transform=transform)

byol_dataloader = DataLoader(byol_dataset, batch_size=64, shuffle=True, num_workers=n_cpu)
byol_module = BYOL(
    hparams={
        "input_dim": 512,
        "hidden_dim": 1024,
        "projection_dim": 256,
        "max_epochs": 3, # used for the cosine scheduler
    },
    model=unet_module.model.encoder,
    batch_type="dict",
)


In [7]:
byol_trainer = pl.Trainer(
    max_epochs=3,
    devices=1,
    accelerator=accelerator,
)

byol_trainer.fit(model=byol_module, train_dataloaders=byol_dataloader) # our model never sees labeled data!

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Fine-tuning our BYOL-boosted-UNET on the rest of the dataset.

In [ ]:
unet_byol_module = UNETModule().from_byol(byol_module)

unet_byol_trainer = pl.Trainer(
    max_epochs=1,
    accelerator=accelerator,
    devices=1,
)

In [ ]:
unet_byol_trainer.fit(unet_byol_module, train_dataloader, valid_dataloader)

In [ ]:
import matplotlib.pyplot as plt

batch = next(iter(test_dataloader))
with torch.no_grad():
    unet_module.eval()
    logits = unet_module(batch["image"].float())

with torch.no_grad():
    unet_byol_module.eval()
    logits_byol = unet_byol_module(batch["image"].float())

pr_masks_unet = logits.sigmoid()
pr_masks_byol = logits_byol.sigmoid()

for image, gt_mask, pr_mask_unet, pr_mask_byol in zip(batch["image"], batch["mask"], pr_masks, pr_masks_byol):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 4, 1)
    plt.imshow(image.numpy().transpose(1, 2, 0))  # convert CHW -> HWC
    plt.title("Image")
    plt.axis("off")

    plt.subplot(1, 4, 2)
    plt.imshow(gt_mask.numpy().squeeze()) # just squeeze classes dim, because we have only one class
    plt.title("Ground truth")
    plt.axis("off")

    plt.subplot(1, 4, 3)
    plt.imshow(pr_mask_unet.numpy().squeeze()) # just squeeze classes dim, because we have only one class
    plt.title("Prediction")
    plt.axis("off")

    plt.subplot(1, 4, 4)
    plt.imshow(pr_mask_byol.numpy().squeeze()) # just squeeze classes dim, because we have only one class
    plt.title("Prediction")
    plt.axis("off")

    plt.show()
     

In [9]:
# import torch.nn as nn

# unet_module = UNETModule(encoder_name="resnet50")
# m = nn.Sequential(*list(unet_module.model.encoder.children()), nn.AdaptiveAvgPool2d((1, 1)))
# m.eval()
# x = torch.rand(1, 3, 32, 32)
# m(x).flatten(start_dim=1).shape

torch.Size([1, 2048])

In [11]:
# import torchvision.models as models

# resnet = models.resnet18(pretrained=False, num_classes=10)
# m = nn.Sequential(*list(resnet.children())[:-1])
# m.eval()
# x = torch.rand(1, 3, 256, 256)
# m(x).flatten(start_dim=1).shape

torch.Size([1, 512])